## <u>Human-in-the-loop</u>

Provide human feedback to a team.

Two main ways to achieve this
1. During .run/.run_stream - provide feedback through UserProxyAgent.
2. Once the run terminates, provide feedback through .run/.run_stream

#### <u>Off topic : code samples on integration with web and UI frameworks</u>

1. <u>[Agentchat + FastAPI](https://github.com/microsoft/autogen/tree/main/python/samples/agentchat_fastapi)</u>
2. <u>[Agentchat + ChainLit](https://github.com/microsoft/autogen/tree/main/python/samples/agentchat_chainlit)</u>
3. <u>[Agentchat + StreamLit](https://github.com/microsoft/autogen/tree/main/python/samples/agentchat_streamlit)</u>

### <u>Providing feedback during the run</u>

As the name goes **UserProxyAgent** is a built-in agent that acts as a proxy for the user and provides feedback.

Create an instance of UPA and include it in the team definition. The team will decide when to call UPA to ask for feedback.

In **RoundRobinGroupChat**, UPA is called by order but in **SelectorGroupChat**, the selector prompt or selector function determines when the UPA is called.

<img src="images/image_14.png" width=700>

Task & TaskResult arrows signify the normal input of task (run/run_stream) and the output TaskResult. The bold arrows show us the flow when the group chat decides to take an opinion from the user and then use that to continue finding the solution.

<u>**Note:**</u> When the UPA is called, until a human replies, the state is suspended mid-execution. If the system crashes at that time, there is no way for the team to resume later.

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"
)
assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)

termination = TextMentionTermination("APPROVE")
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

await Console(team.run_stream(task="Write a 4 line poem about Pune in Marathi."))
await model_client.close()

---------- TextMessage (user) ----------
Write a 4 line poem about Pune in Marathi.
---------- TextMessage (assistant) ----------
पुणे शहर, संस्कृतीची गळामिठी,  
चहा-कॉफीचं, भेटींचं गाणं गाती,  
सार्वजनिक उद्याने, सुखद सावली,  
पुण्याची गोष्ट, हृदयात वसती.  

TERMINATE


Enter your response:  do again


---------- TextMessage (user_promy) ----------
do again
---------- TextMessage (assistant) ----------
पुणे म्हातारं, ज्ञानाचं घर,  
पार्वती पर्वता, निसर्गात मजर,  
छान गप्पा, चहा-भाजी,  
पुण्याची माती, प्रेमभरी सावली.  

TERMINATE


Enter your response:  APPROVE


---------- TextMessage (user_promy) ----------
APPROVE


Done till here